In [1]:
import pandas as pd

comps = pd.read_csv('Competitions.csv')
notebooks = pd.read_csv('final_notebooks.csv')

In [2]:
def unpack_array(arr):
    if isinstance(arr, list):
        return [item for sublist in arr for item in (sublist if isinstance(sublist, list) else [sublist])]
    else:
        return arr

In [25]:
import re

def remove_comments_imports_prints(code):
    # Split the code into lines
    lines = code.split('\n')
    # Create an empty list to store the filtered lines
    filtered_lines = []

    # Flags for block comments
    inside_block_comment = False

    # Iterate over each line
    for line in lines:
        # Handle block comments
        if "'''" in line:
            inside_block_comment = not inside_block_comment
            continue
        if inside_block_comment:
            continue

        # Remove lines that start with an import statement
        if line.strip().startswith("import") or line.strip().startswith("from"):
            continue
        # Remove lines that start with a print statement
        if line.strip().startswith("print"):
            continue
        if line.strip().startswith("plt"):
            continue
        if line.strip().startswith("warnings"):
            continue
        if line.strip().startswith("logging"):
            continue
        if line.strip().startswith("display"):
            continue
        if line.strip().startswith("filterwarnings"):
            continue
        if line.strip().startswith("sns"):
            continue
        if line.strip().startswith("random"):
            continue
        # Remove lines that are comments
        if re.match(r'\s*#', line):
            continue
        if re.match(r'\s*!', line):
            continue
        if re.match(r'\s*%', line):
            continue
        if re.match(r'.*\b(sns)\b.*', line):
            continue
        # Remove lines that contain plotting statements
        if re.match(r'.*\.(plot|show|scatter|hist|imshow|figure|sns)\(', line):
            continue
        # Add the line to the filtered lines list
        filtered_lines.append(line)

    # Join the filtered lines back into a single string
    filtered_code = '\n'.join(filtered_lines)

    # Remove multiple consecutive newline characters
    filtered_code = re.sub(r'\n{2,}', '\n', filtered_code)

    return filtered_code

In [11]:
good_notebooks = notebooks.dropna(subset=['code'])

In [ ]:
good_notebooks['code'] = good_notebooks['code'].apply(lambda x: unpack_array(eval(x)))
good_notebooks['code'][1]

In [26]:
clean_code = []


for i, row in good_notebooks.iterrows():
    full_code = '\n'.join(row['code'])
    clean_code.append(remove_comments_imports_prints(full_code))


good_notebooks['CleanCode'] = clean_code
good_notebooks

/var/folders/zh/dsbs1wbd72z81b85y9td6sx00000gn/T/ipykernel_14566/2942787102.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  good_notebooks['CleanCode'] = clean_code


,Unnamed: 0,CompetitionId,TeamName,PublicLeaderboardRank,PrivateLeaderboardRank,TeamId,UserId,RequestDate,CurrentKernelVersionId,code,CleanCode
0,0,17807,Women Power🇮🇱,2.0,1.0,4240714,109123,01/14/2020,28356656,[import numpy as np \nimport pandas as pd # da...,"\nfor dirname, _, filenames in os.walk('/kaggl..."
1,1,17807,Prevision.io,1.0,3.0,4249216,273819,02/03/2020,29281726,"[from IPython.core.display import display, HTM...",\nrandom.seed(28)\nnp.random.seed(28)\n ...
2,2,17807,EPFL 🇨🇭,6.0,5.0,4393925,1658094,02/15/2020,29200607,"[import os\nfrom scipy.stats import kurtosis, ...","\nfor dirname, _, filenames in os.walk('/kaggl..."
3,3,18005,Samihan Jawalkar,121.0,120.0,4364358,2170525,02/09/2020,28701986,[# This Python 3 environment comes with many h...,"\nfor dirname, _, filenames in os.walk('/kaggl..."
4,4,18627,Y.Nakama,48.0,50.0,4536923,1695531,03/12/2020,30296882,[import os\nimport numpy as np \nimport pandas...,\nwith open('../input/herbarium-2020-fgvc7/nyb...
...,...,...,...,...,...,...,...,...,...,...,...
566,566,65862,NGMI,68.0,2.0,11496974,10543899,01/11/2024,165116389,[import numpy as np\nimport pandas as pd\nimpo...,"\npd.set_option('display.max_rows', 100)\nset_..."
567,567,65862,Ravi Ramakrishnan,127.0,3.0,11491035,8273630,01/09/2024,163291493,[%%time \n\n# Installing select libraries:-\nf...,\n--find-links=/kaggle/input/packageinstallati...
568,568,68479,tdoh86,24.0,2.0,11586690,8063636,02/01/2024,161683762,[# This Python 3 environment comes with many h...,"\nfor dirname, _, filenames in os.walk('/kaggl..."
569,569,68479,Spiritmilk,68.0,8.0,11592560,8060745,02/02/2024,162462887,"[import warnings\nwarnings.filterwarnings(""ign...","\ntrain = pd.read_csv(""/kaggle/input/playgroun..."


In [32]:
cl = []
for index, row in good_notebooks.iterrows():
    cl.append((index, row['CleanCode'].count('\n')))


In [49]:
from heapq import nsmallest, nlargest

print(f'Min: {nsmallest(30, cl, key=lambda item: item[1])}\nMax: {nlargest(5, cl, key=lambda item: item[1])}')

Min: [(108, 0), (305, 0), (320, 0), (331, 0), (369, 0), (507, 0), (537, 0), (280, 1), (371, 1), (459, 1), (520, 1), (67, 2), (432, 2), (499, 2), (534, 2), (168, 3), (463, 3), (157, 4), (307, 4), (325, 4), (479, 5), (480, 6), (283, 7), (487, 7), (175, 8), (512, 9), (114, 10), (242, 11), (271, 11), (169, 12)]
Max: [(446, 2456), (374, 2378), (397, 2366), (510, 2238), (538, 2097)]


In [62]:
final = good_notebooks[good_notebooks['CleanCode'].str.count('\n') < 300]
final = final[good_notebooks['CleanCode'].str.count('\n') > 15]

/var/folders/zh/dsbs1wbd72z81b85y9td6sx00000gn/T/ipykernel_14566/4059254357.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  final = final[good_notebooks['CleanCode'].str.count('\n') > 15]


In [64]:
final.to_csv('notebooks_meta.csv')